In [53]:
# -*- coding: utf-8 -*-
import dataiku
import re
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu

# Read recipe inputs
in_pays = dataiku.Dataset("2018_pays_prepared")
in_pays_df = in_pays.get_dataframe()
in_orgs = dataiku.Dataset("2018_toutes_organisations")
in_orgs_df = in_orgs.get_dataframe()

out_df = in_orgs_df
# renommage des colonnes de sortie
out_df = out_df.rename(columns = {'abbrev':'abbrev_org',
                                  'name':'name_org',
                                 "established": "date de création",
                                  "GPS.value_first": "GPS_siege",
                                  "longitude_first": "longitude_siege",
                                  "latitude_first": "latitude_siege",
                                  "point_palladio_first": "point_palladio",
                                  "siegeLabel.value_first": "nom_siege"
                                 })


# création d'un dictionnaire avec en valeur la liste des orgs auxquelles la clé (nom du pays) appartient
dico_noms_pays_orgs = {}
for row in in_pays.iter_rows():
    dico_noms_pays_orgs[row["name"]]=re.split(r'\\', row["memberships"])

pays_des_orgs = []
for org in [une_org["abbrev"] for une_org in in_orgs.iter_rows()]:
    liste_des_pays = []
    for pays in dico_noms_pays_orgs:
        if org in dico_noms_pays_orgs.get(pays):
            liste_des_pays.append(pays)
    pays_des_orgs.append(liste_des_pays)

out_df["members"]=pays_des_orgs


dico_pop_pays_orgs = {}
for row in in_pays.iter_rows():
    dico_pop_pays_orgs[row["name"]]=row["population"]

population_orgs  = []
for org in pays_des_orgs:
    population_org = 0
    for pays in org:
        population_org += dico_pop_pays_orgs.get(pays)
    population_orgs.append(population_org)

out_df["population_org"]=population_orgs

"""
population_des_orgs = []
for org in [une_org["abbrev"] for une_org in in_orgs.iter_rows()]:
    populations = 0
    for pays in dico_pop_pays_orgs:
        if org in dico_pop_pays_orgs.get(pays):
            populations += dico_pop_pays_orgs.get(pays)
    print(populations)
    pays_des_orgs.append(liste_des_pays)"""


# Write recipe outputs
out = dataiku.Dataset("2018_final")
out.write_with_schema(out_df)

152 rows successfully written (E2FgCRWuFo)
